# Building Fast Queries on a CSV

#### Imagine that you own an online laptop store and want to build a way to answer a few different business questions about your inventory.

#### TIP:Preprocessing data can significantly speed-up an algorithm

## 1.Exploring the data

In [2]:
import csv

In [3]:
with open('laptops.csv', encoding='UTF-8') as file:
    read_file = csv.reader(file)
    laptop_list = list(read_file)
    header = laptop_list[:1]
    rows = laptop_list[1:]
    


In [4]:
header

[['Id',
  'Company',
  'Product',
  'TypeName',
  'Inches',
  'ScreenResolution',
  'Cpu',
  'Ram',
  'Memory',
  'Gpu',
  'OpSys',
  'Weight',
  'Price']]

In [5]:
rows[:5]

[['6571244',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 2.3GHz',
  '8GB',
  '128GB SSD',
  'Intel Iris Plus Graphics 640',
  'macOS',
  '1.37kg',
  '1339'],
 ['7287764',
  'Apple',
  'Macbook Air',
  'Ultrabook',
  '13.3',
  '1440x900',
  'Intel Core i5 1.8GHz',
  '8GB',
  '128GB Flash Storage',
  'Intel HD Graphics 6000',
  'macOS',
  '1.34kg',
  '898'],
 ['3362737',
  'HP',
  '250 G6',
  'Notebook',
  '15.6',
  'Full HD 1920x1080',
  'Intel Core i5 7200U 2.5GHz',
  '8GB',
  '256GB SSD',
  'Intel HD Graphics 620',
  'No OS',
  '1.86kg',
  '575'],
 ['9722156',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '15.4',
  'IPS Panel Retina Display 2880x1800',
  'Intel Core i7 2.7GHz',
  '16GB',
  '512GB SSD',
  'AMD Radeon Pro 455',
  'macOS',
  '1.83kg',
  '2537'],
 ['8550527',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 3.1GHz',
  '8GB',
  '256GB SSD',
  'Intel

## 2. Inventory Class

#### Create an inventory class.
#### Convert the price of each row into an integer. The price is the last column

In [7]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            read_file = csv.reader(file)
            file_list = list(read_file)
            self.header = file_list[:1]
            self.rows = file_list[1:]
            for row in self.rows:       
                row[-1] = int(row[-1])
        

In [8]:
inventory = Inventory('laptops.csv')  
print(inventory.header)               
print(len(inventory.rows))            

[['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']]
1303


##  3. Finding a laptop from the ID

#### Implemented a way to look up a laptop from a given identifier. In this way, when a customer comes to the store with a purchase slip, one can quickly identify the laptop to which it corresponds.



In [9]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]        
        self.rows = rows[1:]
        for row in self.rows:              
            row[-1] = int(row[-1])
            
    def get_laptop_from_id(self, laptop_id):   # step 1
        for row in self.rows:                  # step 2
            if row[0] == laptop_id:
                return row
        return None
            

In [10]:
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## 4. Improving Id Lookups

The algorithm above requires one to look at every single row to find the one that we are looking for (or decide that such a row does not exist). This algorithm has time complexity O(R) where R is the number of rows.

we can solve this problem more efficiently by preprocessing the data. Using a set, we can check in constant time whether a given identifier exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of a set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

In [11]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]        
        self.rows = rows[1:]
        for row in self.rows:              
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:                       
            self.id_to_row[row[0]] = row 
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:                 
            if row[0] == laptop_id:
                return row
        return None   
    
    def get_laptop_from_id_fast(self, laptop_id):   # step 3
        if laptop_id in self.id_to_row:             # step 4
            return self.id_to_row[laptop_id]
        return None
            

In [12]:
inventory = Inventory('laptops.csv')                # step 5
print(inventory.get_laptop_from_id_fast('3362737')) # step 6
print(inventory.get_laptop_from_id_fast('3362736')) # step 7

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## 5. Comparing the Performance

The get_laptop_from_id() method has time complexity O(R) where R is the number of rows. In contrast, the new implementation is time complexity O(1). It does so by using more memory to store the self.id_to_row dictionary and using a bit more time creating an instance (because it needs to create the dictionary).

In [13]:
import time, random

ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)] # step 3

inventory = Inventory('laptops.csv')                                # step 4

total_time_no_dict = 0                                              # step 5
for identifier in ids:                                              # step 6
    start = time.time()                                             # step 6.1
    inventory.get_laptop_from_id(identifier)                        # step 6.2
    end = time.time()                                               # step 6.3
    total_time_no_dict += end - start                               # step 6.4
    
total_time_dict = 0                                                 # step 7
for identifier in ids:                                              # step 8
    start = time.time()                                             # step 8.1
    inventory.get_laptop_from_id_fast(identifier)                   # step 8.2
    end = time.time()                                               # step 8.3
    total_time_dict += end - start                                  # step 8.4
    

In [14]:
print(total_time_no_dict)                                           # step 9
print(total_time_dict)

1.5008983612060547
0.00555872917175293


We can see a significant improve in performance. If we divide 1.5 by 0.006 we see that the new method is about 250 times faster for this input size.

## 6. Two Laptop Promotion

Sometimes, your store offers a promotion where you give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

Before issuing a gift card for D dollars, you want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

Write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

In [15]:
class Inventory():                    
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]        
        self.rows = rows[1:]
        for row in self.rows:              
            row[-1] = int(row[-1])
        self.id_to_row = {}                        
        for row in self.rows:                       
            self.id_to_row[row[0]] = row 
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:                 
            if row[0] == laptop_id:
                return row
        return None   
    
    def get_laptop_from_id_fast(self, laptop_id):  
        if laptop_id in self.id_to_row:           
            return self.id_to_row[laptop_id]
        return None

    def check_promotion_dollars(self, dollars):    # step 1
        for row in self.rows:                      # step 2
            if row[-1] == dollars:
                return True
        for row1 in self.rows:                     # step 3
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False                            

In [16]:
inventory = Inventory('laptops.csv')               # step 5
print(inventory.check_promotion_dollars(1000))     # step 6
print(inventory.check_promotion_dollars(442)) 

True
False


## Optimizing Laptop promotion

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.

In [17]:
class Inventory():                    
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]        
        self.rows = rows[1:]
        for row in self.rows:              
            row[-1] = int(row[-1])
        self.id_to_row = {}                        
        for row in self.rows:                       
            self.id_to_row[row[0]] = row
        self.prices = set()                          # step 1
        for row in self.rows:                        # step 2
            self.prices.add(row[-1])
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:                 
            if row[0] == laptop_id:
                return row
        return None   
    
    def get_laptop_from_id_fast(self, laptop_id):  
        if laptop_id in self.id_to_row:           
            return self.id_to_row[laptop_id]
        return None

    def check_promotion_dollars(self, dollars):    
        for row in self.rows:                   
            if row[-1] == dollars:
                return True
        for row1 in self.rows:                  
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False                        
    
    def check_promotion_dollars_fast(self, dollars): # step 3
        if dollars in self.prices:                   # step 4
            return True
        for price in self.prices:                    # step 5
            if dollars - price in self.prices:
                return True
        return False                                 # step 6

In [18]:
inventory = Inventory('laptops.csv')                 # step 7
print(inventory.check_promotion_dollars_fast(1000))  # step 8
print(inventory.check_promotion_dollars_fast(442))   # step 9

True
False


## Comparing Promotion Functions

In [19]:
prices = [random.randint(100, 5000) for _ in range(100)] # step 1

inventory = Inventory('laptops.csv')                     # step 2

total_time_no_set = 0                                    # step 3
for price in prices:                                     # step 4
    start = time.time()                                  # step 4.1
    inventory.check_promotion_dollars(price)             # step 4.2
    end = time.time()                                    # step 4.3
    total_time_no_set += end - start                     # step 4.4
    
total_time_set = 0                                       # step 5
for price in prices:                                     # step 6
    start = time.time()                                  # step 6.1
    inventory.check_promotion_dollars_fast(price)        # step 6.2
    end = time.time()                                    # step 6.3
    total_time_set += end - start                        # step 6.4
    
print(total_time_no_set)                                 # step 7
print(total_time_set)

2.518876552581787
0.0009846687316894531


We can see a significant improve in performance

## Finding Laptops Within a Budget

Given a budget of D dollars, find all laptops whose price it at most D.

If we sort all laptops by price, we can use binary search to identify the first laptop in the sorted list with a price larger than D. We need to make sure that our binary search finds the first one on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than D dollars:

In [20]:
def row_price(row):
    return row[-1]

class Inventory():                    
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]        
        self.rows = rows[1:]
        for row in self.rows:              
            row[-1] = int(row[-1])
        self.id_to_row = {}                        
        for row in self.rows:                       
            self.id_to_row[row[0]] = row
        self.prices = set()                          
        for row in self.rows:                        
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key=row_price) # Step 1
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:                 
            if row[0] == laptop_id:
                return row
        return None   
    
    def get_laptop_from_id_fast(self, laptop_id):  
        if laptop_id in self.id_to_row:           
            return self.id_to_row[laptop_id]
        return None

    def check_promotion_dollars(self, dollars):    
        for row in self.rows:                   
            if row[-1] == dollars:
                return True
        for row1 in self.rows:                  
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False                        
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:                   
            return True
        for price in self.prices:                    
            if dollars - price in self.prices:
                return True
        return False                                
    
    def find_laptop_with_price(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            value = self.rows_by_price[range_middle][-1]
            if value == target_price:                            
                return range_middle                        
            elif value < target_price:                           
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle - 1 
        if self.rows_by_price[range_start][-1] != target_price:                  
            return -1                                      
        return range_start
    
    def find_first_laptop_more_expensive(self, target_price): # Step 2
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start

inventory = Inventory('laptops.csv')                     # Step 3            
print(inventory.find_first_laptop_more_expensive(1000))  # Step 4
print(inventory.find_first_laptop_more_expensive(10000)) # Step 5

683
-1
